<a href="https://colab.research.google.com/github/abomhold/Catagorical-Regression-Analysis/blob/master/regression_tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This notebook details the comparision between decision tree regression, and the KNN regression of the tree trunk temperature modeling problem.

Assume $Y=f(\mathbf{X})+\epsilon$

Here $Y:$ response

(temperature in tree trunk)

$\mathbf{X}=[X_1, ..., X_n]:$ features, or predictors

(weather condition)

$\epsilon:$ random error term (independent of $\mathbf{X}$)

## Data explanation
We model the temperature of a tree trunk using various regression methods, based on weather conditions and measurements of the bark and core of the tree trunks.

We use data collected onsite in Brazil, over a 7 day period, in August of 2022. These data include weather data, as well as temperature data consisting of core and bark temperatures, with two mid-trunk temperatures as validation for our model.

We used a developed wireless sensor device that can measure temperature in 10 different position (direction and height) around the tree trunk and 3 different depths. Two spreadsheets were populated with 1 week long of data both from the tree and from our weather station in Brazil.

The Tree Data Spreadsheet has the following notation:

1. DXX@YY, where D = direction (N = North, S = South, E = East, W =West), XX = depth of the temperature sensor from the tree core, and YY = height of the temperature sensor. For example: S4.5cm@1m = sensor to the South 4.5 cm depth at 1 m height.
2. Only the external sensor has a different notation, namely W\_Ext\_Temp\@ 3.5m = to the West at 3.5 m height.

The weather station (Modelo D140193 of Ammonit Measurement GmbH) is located near the tree. The weather station Spreadsheet has the following parameters:

  1. ``Anemometer;win\_speed;Avg (m/s)'': Average wind speed in m/s measured by a anemometer
  2. ``Wind Vane TMR;wind\_direction;Avg ($^{\circ}$)'': Average wind direction in $^{\circ}$ measured by a Wind Vane
  3. ``Hygro/Thermo;humidity;Avg (\%)'': Average  humidity in \% measured by a Hygrometer
  4. ``Hygro/Thermo;temperature;Avg ($^{\circ}$C)'': Average temperature in $^{\circ}$C measured by a Hygrometer
  5. ``Barometer;air\_pressure;Avg (hPa)'': Average air pressure in hPa   measured by a Barometer
  6. ``DNI (Direct Normal Irradiance) Pyrheliometer;solar\_DNI;Avg (W/m$^2$)'':  Average Direct Normal Irradiance in W/m$^2$ measured by a Pyrheliometer.

The Direct Normal Irradiance (DNI) is a measurement of the Solar Irradiance. The Global Irradiance is the sum of the DNI and the Diffuse Irradiance. DNI is a fraction of the global solar irradiation, and is the most important parameter in a solar plant installation. So, we use of DNI to measure solar radiation in this model.

Weather data is collected every 10 minutes, starting at 00:00; temperature data is collected every 3 minutes, starting at 00:01. It is reasonable to assume weather parameters and tree temperatures are continuous functions of time, so we linearly interpret the collected data to obtain measurements of the same length for our regression analysis.

The radius of the mango tree is $r = 0.135$ m, and consider a 24 hour period. We use $n_t$ to denote time steps over time, and $n_x$ grid points in space, and these are parameters used in the linear interpolation of temperature and weather data.

In [3]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import sys
print("Python 3 version is", sys.version)
import matplotlib
print("Matplotlib version is", matplotlib.__version__)
print("Numpy version is", np.__version__)
print("Pandas version is", pd.__version__)

Python 3 version is 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]
Matplotlib version is 3.10.0
Numpy version is 1.26.4
Pandas version is 2.2.2


In [4]:
# from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor, AdaBoostRegressor, BaggingRegressor, IsolationForest

from sklearn.metrics import mean_squared_error, mean_absolute_error
# from sklearn import preprocessing
import sklearn
print("scikit learn version is", sklearn. __version__)

scikit learn version is 1.6.1


In [5]:
# define parameter
n_t = 450
radius = 0.135 # unit m
time = np.linspace(0, 24, n_t, endpoint = False)

In [6]:
# # for binder's jupyterlab
# !pip3 install openpyxl

In [7]:
colnames = ['temp_datetime', 's45_1', 'e9_1', 'n135_1','e45_2', 'n9_2', 'w135_2', 'n45_3', 'w9_3','s135_3', 'w_ext_35']
url1 = "https://raw.githubusercontent.com/yajuna/linearRegression/master/Tree_Temp_Values_AUG21_to_AUG28_2022.xlsx"
dataTemp = pd.read_excel(url1,names=colnames)
dataTemp['temp_datetime'] = pd.to_datetime(dataTemp['temp_datetime'])
dataTemp = dataTemp.set_index('temp_datetime')

In [8]:
train_range = dataTemp.loc['2022-08-21':'2022-08-21 23:59:59']

test_range = dataTemp.loc['2022-08-22':'2022-08-22 23:59:59']

train_temp_size = len(train_range.index)
# linear interpolate the measured temperature
train_coreTemp = np. interp(time, np.linspace(0,24,train_temp_size),train_range.s135_3)
train_midTemp1 = np. interp(time, np.linspace(0,24,train_temp_size),train_range.w9_3)
train_midTemp2 = np. interp(time, np.linspace(0,24,train_temp_size),train_range.n45_3)
train_barkTemp = np. interp(time, np.linspace(0,24,train_temp_size),train_range.w_ext_35)

test_temp_size = len(test_range.index)
test_coreTemp = np. interp(time, np.linspace(0,24,test_temp_size),test_range.s135_3)
test_midTemp1 = np. interp(time, np.linspace(0,24,test_temp_size),test_range.w9_3)
test_midTemp2 = np. interp(time, np.linspace(0,24,test_temp_size),test_range.n45_3)
test_barkTemp = np. interp(time, np.linspace(0,24,test_temp_size),test_range.w_ext_35)

print("measured temperature data read")
print(train_range)

print(test_range)

measured temperature data read
                     s45_1   e9_1  n135_1  e45_2   n9_2  w135_2  n45_3   w9_3  \
temp_datetime                                                                   
2022-08-21 00:01:51  25.18  26.25   24.62  25.62  26.12   26.00  25.25  26.00   
2022-08-21 00:05:07  25.18  26.18   24.62  25.68  26.12   26.00  25.25  26.00   
2022-08-21 00:08:23  25.12  26.18   24.62  25.62  26.12   26.00  25.18  25.93   
2022-08-21 00:11:38  25.18  26.18   24.56  25.62  26.06   26.00  25.18  26.00   
2022-08-21 00:14:54  25.12  26.18   24.62  25.62  26.06   26.00  25.18  26.00   
...                    ...    ...     ...    ...    ...     ...    ...    ...   
2022-08-21 23:46:08  25.18  26.43   25.06  25.87  26.68   26.56  25.56  26.37   
2022-08-21 23:49:24  25.18  26.43   25.06  25.87  26.68   26.56  25.56  26.37   
2022-08-21 23:52:40  25.18  26.43   25.06  25.87  26.68   26.56  25.50  26.37   
2022-08-21 23:55:56  25.12  26.43   25.00  25.87  26.62   26.50  25.50  26.37 

In [9]:
colnames = ['weather_datetime', 'wind_speed', 'wind_direction', 'humidity', 'air_temperature', 'air_pressure', 'solar_DNI']
url2 = "https://raw.githubusercontent.com/yajuna/linearRegression/master/Weather_Station_AUG21_to_AUG28_2022.xlsx"
dataWeather = pd.read_excel(url2,names=colnames)
dataWeather['weather_datetime'] = pd.to_datetime(dataWeather['weather_datetime'])
dataWeather = dataWeather.set_index('weather_datetime')

In [10]:
train_weather = dataWeather.loc['2022-08-21':'2022-08-21 23:59:59']

test_weather = dataWeather.loc['2022-08-22':'2022-08-22 23:59:59']

train_weather_size = len(train_weather.index)

train_airTemp = np.interp(time, np.linspace(0,24,train_weather_size),train_weather.air_temperature)
train_windSpeed = np.interp(time, np.linspace(0,24,train_weather_size),train_weather.wind_speed)
train_solar = np.interp(time, np.linspace(0,24,train_weather_size),train_weather.solar_DNI)
train_humidity = np.interp(time, np.linspace(0,24,train_weather_size),train_weather.humidity)
train_airPressure = np.interp(time, np.linspace(0,24,train_weather_size),train_weather.air_pressure)

test_weather_size = len(test_weather.index)

test_airTemp = np.interp(time, np.linspace(0,24,test_weather_size),test_weather.air_temperature)
test_windSpeed = np.interp(time, np.linspace(0,24,test_weather_size),test_weather.wind_speed)
test_solar = np.interp(time, np.linspace(0,24,test_weather_size),test_weather.solar_DNI)
test_humidity = np.interp(time, np.linspace(0,24,test_weather_size),test_weather.humidity)
test_airPressure = np.interp(time, np.linspace(0,24,test_weather_size),test_weather.air_pressure)

print(train_weather)

print(test_weather)

                     wind_speed  wind_direction  humidity  air_temperature  \
weather_datetime                                                             
2022-08-21 00:00:00      2.1758        195.8894   73.0005          23.8425   
2022-08-21 00:10:00      2.0381        195.6512   73.2226          23.5854   
2022-08-21 00:20:00      1.7478        193.6007   73.5104          23.3592   
2022-08-21 00:30:00      1.6064        188.9758   73.7139          23.1699   
2022-08-21 00:40:00      1.3614        199.9231   73.9565          22.9508   
...                         ...             ...       ...              ...   
2022-08-21 23:10:00      1.0138        213.2707   87.4109          22.7085   
2022-08-21 23:20:00      1.4568        205.0766   87.7499          22.6212   
2022-08-21 23:30:00      1.5097        197.3073   88.0380          22.6066   
2022-08-21 23:40:00      1.5505        194.7728   88.2570          22.6291   
2022-08-21 23:50:00      2.1849        191.2897   88.4429       

## Prepare training and testing data

The training data is collected on August 21, and the testing data is collected on August 22, as well as August 26

In [11]:
#include depth as training variable
# n = 0 for 9cm; n = 1 for 4.5cm
n = 1
depth_list = [0.09/radius, 0.045/radius]

# training Y temp
Ytrain_list = [train_midTemp1, train_midTemp2]

Ytest_list = [test_midTemp1, test_midTemp2]

# depth for training is depth1 or depth2
depth = np.ones(train_coreTemp.shape) * depth_list[n]

# Training data in x; training data in y
Xtrain = [1./train_humidity, train_airTemp, train_windSpeed, train_solar, depth, train_coreTemp, train_barkTemp]

X_train = np.array(Xtrain).T
Y_train = Ytrain_list[n]

# testing data in x, testing data in y.
ic_test = np.ones(test_coreTemp.shape) * Ytest_list[n][0]
Xtest = [1./test_humidity, test_airTemp, test_windSpeed, test_solar, depth, test_coreTemp, test_barkTemp]
X_test = np.array(Xtest).T
Y_test = Ytest_list[n]

## In the following, we look at regression with the K nearest neighbors (KNN) as well as Decision trees.

Both methods are implemented in Python, with the `scikit learn` library.

## The following two cells are for decision tree regression

In [12]:
# Fit regression model with decision tree
dt_regressor = DecisionTreeRegressor(max_depth=4)
dt_regressor.fit(X_train, Y_train)

DecisionTreeRegressor(max_depth=4)

In [13]:
# Fit regression model with random forest
rf_regressor = RandomForestRegressor(n_estimators = 5, random_state = 0)
rf_regressor.fit(X_train, Y_train)

RandomForestRegressor(n_estimators=5, random_state=0)

In [14]:
# Fit regression model with random forest
ex_rf_regressor = ExtraTreesRegressor(n_estimators = 5, random_state = 0)
ex_rf_regressor.fit(X_train, Y_train)

ExtraTreesRegressor(n_estimators=5, random_state=0)

In [ ]:
# Fit regression model with random forest
gb_rf_regressor = GradientBoostingRegressor(n_estimators = 5, random_state = 0)
gb_rf_regressor.fit(X_train, Y_train)

In [ ]:
# Fit regression model with random forest
ab_rf_regressor = AdaBoostRegressor(n_estimators = 5, random_state = 0)
ab_rf_regressor.fit(X_train, Y_train)

In [ ]:
# Fit regression model with random forest
br_rf_regressor = BaggingRegressor(n_estimators = 5, random_state = 0)
br_rf_regressor.fit(X_train, Y_train)

In [ ]:
# Fit regression model with random forest
is_rf_regressor = IsolationForest(n_estimators = 5, random_state = 0)
is_rf_regressor.fit(X_train, Y_train)

In [ ]:
# prediction with the above regression models

y_1 = dt_regressor.predict(X_test)
y_2 = rf_regressor.predict(X_test)
y_3 = ex_rf_regressor.predict(X_test)
y_4 = gb_rf_regressor.predict(X_test)
y_5 = ab_rf_regressor.predict(X_test)
y_6 = br_rf_regressor.predict(X_test)
y_7 = is_rf_regressor.predict(X_test)

In [ ]:
# visualization
print ("Absolute error is", np.max(np.abs(Y_test - y_1)), ", Relative error is", np.max(np.abs((Y_test - y_1)/Y_test)))


fig0, axs = plt.subplots(3, sharex=True, sharey=False)
fig0.suptitle('Errors with a tree')
axs[0].plot(time, Y_test, 'b-', label='Measured values (K)')
axs[0].plot(time, y_1, 'r-', label='Predicted values (K)')
axs[1].plot(time, Y_test - y_1, 'b-', label='Absolute error (K)')
axs[2].plot(time, (Y_test - y_1)/Y_test, 'b-', label='Relative error (%)')
axs[0].legend(loc='lower left', fontsize = 'x-small')
axs[1].legend(loc='lower left', fontsize = 'x-small')
axs[2].legend(loc='lower left', fontsize = 'x-small')
plt.xlabel('time (hrs)', fontsize=10)
# fig0.savefig("shallow_decisionTree.eps")
plt.show()

In [ ]:
# visualization
print ("Absolute error is", np.max(np.abs(Y_test - y_2)), ", Relative error is", np.max(np.abs((Y_test - y_2)/Y_test)))


fig, axs = plt.subplots(3, sharex=True, sharey=False)
fig.suptitle('Errors with a forest')
axs[0].plot(time, Y_test, 'b-', label='Measured values (K)')

axs[0].plot(time, y_2, 'r-', label='Predicted values (K)')
axs[1].plot(time, Y_test - y_2, 'b-', label='Absolute error (K)')
axs[2].plot(time, (Y_test - y_2)/Y_test, 'b-', label='Relative error (%)')
axs[0].legend(loc='lower left', fontsize = 'x-small')
axs[1].legend(loc='lower left', fontsize = 'x-small')
axs[2].legend(loc='lower left', fontsize = 'x-small')
plt.xlabel('time (hrs)', fontsize=10)
# fig.savefig("deep_decisionTree.eps")
plt.show()

In [ ]:
# visualization
print ("Absolute error is", np.max(np.abs(Y_test - y_3)), ", Relative error is", np.max(np.abs((Y_test - y_3)/Y_test)))


fig, axs = plt.subplots(3, sharex=True, sharey=False)
fig.suptitle('Errors with an ExtraTreesForest')
axs[0].plot(time, Y_test, 'b-', label='Measured values (K)')

axs[0].plot(time, y_3, 'r-', label='Predicted values (K)')
axs[1].plot(time, Y_test - y_3, 'b-', label='Absolute error (K)')
axs[2].plot(time, (Y_test - y_3)/Y_test, 'b-', label='Relative error (%)')
axs[0].legend(loc='lower left', fontsize = 'x-small')
axs[1].legend(loc='lower left', fontsize = 'x-small')
axs[2].legend(loc='lower left', fontsize = 'x-small')
plt.xlabel('time (hrs)', fontsize=10)
# fig.savefig("deep_decisionTree.eps")
plt.show()

In [ ]:
# visualization
print ("Absolute error is", np.max(np.abs(Y_test - y_4)), ", Relative error is", np.max(np.abs((Y_test - y_4)/Y_test)))


fig, axs = plt.subplots(3, sharex=True, sharey=False)
fig.suptitle('Errors with an GradientBoostForest')
axs[0].plot(time, Y_test, 'b-', label='Measured values (K)')

axs[0].plot(time, y_4, 'r-', label='Predicted values (K)')
axs[1].plot(time, Y_test - y_4, 'b-', label='Absolute error (K)')
axs[2].plot(time, (Y_test - y_4)/Y_test, 'b-', label='Relative error (%)')
axs[0].legend(loc='lower left', fontsize = 'x-small')
axs[1].legend(loc='lower left', fontsize = 'x-small')
axs[2].legend(loc='lower left', fontsize = 'x-small')
plt.xlabel('time (hrs)', fontsize=10)
# fig.savefig("deep_decisionTree.eps")
plt.show()

In [ ]:
# visualization
print ("Absolute error is", np.max(np.abs(Y_test - y_5)), ", Relative error is", np.max(np.abs((Y_test - y_5)/Y_test)))


fig, axs = plt.subplots(3, sharex=True, sharey=False)
fig.suptitle('Errors with an AdaBoostForest')
axs[0].plot(time, Y_test, 'b-', label='Measured values (K)')

axs[0].plot(time, y_5, 'r-', label='Predicted values (K)')
axs[1].plot(time, Y_test - y_5, 'b-', label='Absolute error (K)')
axs[2].plot(time, (Y_test - y_5)/Y_test, 'b-', label='Relative error (%)')
axs[0].legend(loc='lower left', fontsize = 'x-small')
axs[1].legend(loc='lower left', fontsize = 'x-small')
axs[2].legend(loc='lower left', fontsize = 'x-small')
plt.xlabel('time (hrs)', fontsize=10)
# fig.savefig("deep_decisionTree.eps")
plt.show()

In [ ]:
# visualization
print ("Absolute error is", np.max(np.abs(Y_test - y_6)), ", Relative error is", np.max(np.abs((Y_test - y_6)/Y_test)))


fig, axs = plt.subplots(3, sharex=True, sharey=False)
fig.suptitle('Errors with an BaggingForest')
axs[0].plot(time, Y_test, 'b-', label='Measured values (K)')

axs[0].plot(time, y_6, 'r-', label='Predicted values (K)')
axs[1].plot(time, Y_test - y_6, 'b-', label='Absolute error (K)')
axs[2].plot(time, (Y_test - y_6)/Y_test, 'b-', label='Relative error (%)')
axs[0].legend(loc='lower left', fontsize = 'x-small')
axs[1].legend(loc='lower left', fontsize = 'x-small')
axs[2].legend(loc='lower left', fontsize = 'x-small')
plt.xlabel('time (hrs)', fontsize=10)
# fig.savefig("deep_decisionTree.eps")
plt.show()

In [ ]:
# visualization
print ("Absolute error is", np.max(np.abs(Y_test - y_7)), ", Relative error is", np.max(np.abs((Y_test - y_7)/Y_test)))


fig, axs = plt.subplots(3, sharex=True, sharey=False)
fig.suptitle('Errors with an IsolationTreesForest')
axs[0].plot(time, Y_test, 'b-', label='Measured values (K)')

axs[0].plot(time, y_7, 'r-', label='Predicted values (K)')
axs[1].plot(time, Y_test - y_7, 'b-', label='Absolute error (K)')
axs[2].plot(time, (Y_test - y_7)/Y_test, 'b-', label='Relative error (%)')
axs[0].legend(loc='lower left', fontsize = 'x-small')
axs[1].legend(loc='lower left', fontsize = 'x-small')
axs[2].legend(loc='lower left', fontsize = 'x-small')
plt.xlabel('time (hrs)', fontsize=10)
# fig.savefig("deep_decisionTree.eps")
plt.show()

## In the following, we look at KNN regression model
We experiment with different values of $K$ and observe bias, as well as vriance.


In [ ]:
n_neighbors = 4
# weights = "uniform", "distance"

knn = KNeighborsRegressor(n_neighbors, weights="distance")
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)

In [ ]:
# visualization
print ("Absolute error is", np.max(np.abs(Y_test - Y_pred)), ", Relative error is", np.max(np.abs((Y_test - Y_pred)/Y_test)))

fig1, axs = plt.subplots(3, sharex=True, sharey=False)
fig1.suptitle('Measured temperature, predicted temperature, absolute, and relative error with KNN')
axs[0].plot(time, Y_test, 'b-', label='Measured values (K)')
axs[0].plot(time, Y_pred, 'r-', label='Predicted values (K)')
axs[1].plot(time, Y_test - Y_pred, 'b-', label='Absolute error (K)')
axs[2].plot(time, (Y_test - Y_pred)/Y_test, 'b-', label='Relative error (%)')

axs[0].legend(loc='lower left', fontsize = 'x-small')
axs[1].legend(loc='lower left', fontsize = 'x-small')
axs[2].legend(loc='lower left', fontsize = 'x-small')
plt.xlabel('time (hrs)', fontsize=10)
# fig.savefig("deep_decisionTree.eps")
plt.show()